In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

from utils import (
    detrend, monthly_anomalies,
    standardize, reg_slope, corr_with_ttest,
)

### D20 

In [2]:
t_grid_file1 = xr.open_zarr('data/ORCA025.L46-KFS006_TroPA.zarr')

In [3]:
eq = dict(lat=0,lon=slice(140,260))

In [4]:
d20_mon_anom = monthly_anomalies(
    t_grid_file1.d20.sel(**eq)
)
d20_mon_anom.attrs.update({"long_name": "D20 anomaly", "units": "m", "positive": "down"})

d20_eq = detrend(
    d20_mon_anom.fillna(0.).compute()
).rename('D20')

In [5]:
mean_d20 = t_grid_file1.d20.sel(**eq).mean('time')

### Sea Temperature

In [6]:
data = '/data/user/mclaus/data/ORCA025.L46-KFS006_TroPA/'
t_grid_file = xr.open_zarr(data + 't_grid')
t_grid_file = t_grid_file.assign_coords(
    coords={
        'x': (t_grid_file.nav_lon[0,:] + 360) % 360,
        'y': t_grid_file.nav_lat[:,0]
    }
).rename(
   {
       'time_counter':'time',   
       'x':'lon',
       'y':'lat', 
   }
)

/tmp/ipykernel_4040065/627937798.py:2: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  t_grid_file = xr.open_zarr(data + 't_grid')


In [7]:
ste_o = t_grid_file.votemper.isel(deptht=slice(0,16)).sel(**eq)

In [8]:
ste_mon_anom = monthly_anomalies(ste_o)
ste_mon_anom.attrs.update({"long_name": "Sea Temperature anomaly", "units": "°C"})

In [9]:
ste_eq = detrend(ste_mon_anom.compute()).rename('STE')

In [10]:
corr, c_low, c_up = corr_with_ttest(
    d20_eq,ste_eq, dim="time"
)

## Write output

In [11]:
ds = xr.Dataset(
    dict(
        corr=corr,
        mean_d20=mean_d20,
    )
)

In [12]:
!rm -rf data_figS2.zarr
ds.to_zarr("data_figS2.zarr");